<a href="https://colab.research.google.com/github/geraw/ProvengoDemo/blob/main/ProvegoTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [26]:
!wget -nc https://downloads.provengo.tech/unix-dist/deb/Provengo-deb.deb
!apt-get install ./Provengo-deb.deb

'wget' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
%pip install flask

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


# System Under Test

In [28]:
import socket
import random

host = socket.gethostbyname(socket.gethostname())
print(f"{host=}")

host='192.168.56.1'


In [ ]:
from flask import Flask, request, jsonify
import threading
import time
import os
import sys
import json
import importlib
import signal

port = random.randint(1024, 65535)
url = f"http://{host}:{port}"

# Create Flask app
app = Flask(__name__)
#run_with_ngrok(app)  # Allows access from outside

# In-memory data store (replace with a database in a real application)
users = []
loans = []
holds = []
books = {}

# A rute for testing that resets the database and loads initial data
@app.route('/reset', methods=['POST'])
def reset_database():
    global users, loans, holds, books
    
    # Clear all data
    users = []
    loans = []
    holds = []
    books = {}
    
    # If initial data is provided, load it
    if request.is_json:
        data = request.get_json()
        if 'users' in data:
            users.extend(data['users'])
        if 'loans' in data:
            loans.extend(data['loans'])
        if 'holds' in data:
            holds.extend(data['holds'])
        if 'books' in data:
            books.update(data['books'])
    
    return jsonify({
        'message': 'Database reset',
        'status': {
            'users': len(users),
            'loans': len(loans),
            'holds': len(holds),
            'books': len(books)
        }
    }), 200


# --- User Routes ---
@app.route('/users', methods=['POST'])
def add_user():
    user = request.get_json()

    if 'id' not in user:
        print("Error: Attempt to add user without id")
        return jsonify({'error': 'user id is required'}), 400
    if  user.get('id') in [u.get('id') for u in users]:
        print("Error: Attempt to add duplicate user")
        return jsonify({'error': 'User already exists'}), 400
    else:
      users.append(user)
      return jsonify({'message': 'User Added', 'user': user}), 201

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    users = [user for user in users if user.get('id') != user_id]
    return jsonify({'message': 'User deleted'}), 200

@app.route('/users', methods=['GET'])
def search_users():
    query = request.args.get('q', '').lower()
    results = [user for user in users if query in str(user).lower()] if query else users
    return jsonify(results)

# --- Loan Routes ---
@app.route('/loans', methods=['POST'])
def add_loan():
    loan = request.get_json()
    loans.append(loan)
    return jsonify({'message': 'Loan added', 'loan': loan}), 201

@app.route('/loans/<int:loan_id>', methods=['DELETE'])
def delete_loan(loan_id):
    global loans
    loans = [loan for loan in loans if loan.get('id') != loan_id]
    return jsonify({'message': 'Loan deleted'}), 200

@app.route('/loans', methods=['GET'])
def search_loans():
    query = request.args.get('q', '').lower()
    results = [loan for loan in loans if query in str(loan).lower()] if query else loans
    return jsonify(results)

# --- Hold Routes ---
@app.route('/holds', methods=['POST'])
def add_hold():
    hold = request.get_json()
    holds.append(hold)
    return jsonify({'message': 'Hold added', 'hold': hold}), 201

@app.route('/holds/<int:hold_id>', methods=['DELETE'])
def delete_hold(hold_id):
    global holds
    holds = [hold for hold in holds if hold.get('id') != hold_id]
    return jsonify({'message': 'Hold deleted'}), 200

@app.route('/holds', methods=['GET'])
def search_holds():
    query = request.args.get('q', '').lower()
    results = [hold for hold in holds if query in str(hold).lower()] if query else holds
    return jsonify(results)

# --- Book Routes ---
@app.route('/books', methods=['GET', 'POST'])
def manage_books():
    if request.method == 'GET':
        return jsonify(books)
    elif request.method == 'POST':
        book_data = request.get_json()
        book_id = book_data.get('id')
        if book_id:
            books[book_id] = book_data
            return jsonify({'message': 'Book added'}), 201
        return jsonify({'error': 'Invalid book data'}), 400


threading.Thread(target=app.run, kwargs={'host':host,'port':port}).start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.56.1:30889
Press CTRL+C to quit
192.168.56.1 - - [15/Mar/2025 09:31:05] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 09:31:05] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 09:31:15] "POST /users HTTP/1.1" 201 -


In [ ]:
with open("reset.py", "w") as f:
    f.write(f"""
import requests
test_data = {{
    'users': [{{ 'id': 1, 'name': 'Test User' }}],
    'books': {{'1': {{ 'id': '1', 'title': 'Test Book' }}}}}}
requests.post('{url}/reset', json=test_data)
    """)

In [ ]:
import requests

test_data = {
    "users": [{"id": 1, "name": "Test User"}],
    "books": {"1": {"id": "1", "title": "Test Book"}}
}
requests.post(url+'/reset', json=test_data)

Writing reset.py


In [75]:
import requests

try:
    response = requests.post(url+"/users", json = {"id": 225, "name": "Dror"})
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    print("Successfully accessed the server!")
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Error accessing the server: {e}")


Successfully accessed the server!
Response: {'message': 'User Added', 'user': {'id': 225, 'name': 'Dror'}}


# Test model

In [64]:
!provengo --batch-mode create BookStoreDemo

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] ERR c:\temp\ProvengoDemo\BookStoreDemo Folder already exist. delete the existing folder or create new project with new name.
[Main ] ERR Terminating because of previous errors. See logs.
[Main ] ERR 40 BadRequest


In [65]:
import os

with open("BookStoreDemo/spec/js/_constants.js", "w") as f:
  f.write(f"""
const {host=};
const {port=};
""")

In [67]:
%%writefile BookStoreDemo/spec/js/hello-world.js

const svc = new RESTSession("http://"+host+":"+port, "provengo basedclient", {
    headers: {"Content-Type":"application/json"}
});

function validation_func(response){
  const respObj = JSON.parse(response.body);
  if ( respObj.message != "User Added" ) {
      pvg.fail("user creation failed: " + respObj.message);
  } else {
      pvg.log.info(`response is: ${JSON.stringify(respObj)}`);
  }
}


bthread("API User", function(){
    svc.get("/users", {
      callback: function(response){
        const respObj = JSON.parse(response.body);
        pvg.log.info(`response is: ${JSON.stringify(respObj)}`);
      }
    })

    svc.post("/users", {
        body: JSON.stringify({
            id: 111,
            name: "John Doe",
        }),
        callback: validation_func
    });

    svc.get("/users")
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [79]:
!provengo run BookStoreDemo --before="python reset.py"

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] INFO Input path: c:\temp\ProvengoDemo\BookStoreDemo
[RUN  ] INFO Preparing to run the model
[RUN>BUILD] INFO hello-world.js: Library 'REST' summoned automatically. Add "//@provengo summon rest" to the top of the file to explicitly summon it. Set the auto-summon configuration key to false to disable this behavior
[RUN>random>Before Test] INFO Running Before Test command: python reset.py
[RUN>random>Before Test] ERR Traceback (most recent call last):

[RUN>random>Before Test] ERR   File "c:\temp\ProvengoDemo\reset.py", line 7, in <module>

[RUN>random>Before Test] ERR     requ

In [49]:
%%writefile BookStoreDemo/spec/js/hello-world.js

const svc = new RESTSession("http://"+host+":"+port, "provengo basedclient", {
    headers: {"Content-Type":"application/json"}
});


bthread("Test User Flow", function() {
    // Test Get Users
    svc.get("/users", {
        callback: function(response) {
            const respObj = JSON.parse(response.body);
            pvg.log.info(`Initial users: ${JSON.stringify(respObj)}`);
        }
    });

    // Test Create User
    svc.post("/users", {
        body: JSON.stringify({
            id: 111,
            name: "John Doe"
        }),
        callback: function(response) {
            const respObj = JSON.parse(response.body);
            if (respObj.message !== "User Added") {
                pvg.fail("User creation failed: " + respObj.message);
            }
            pvg.log.info(`Create user response: ${JSON.stringify(respObj)}`);
        }
    });

    // Test Get Updated Users
    svc.get("/users", {
        callback: function(response) {
            const respObj = JSON.parse(response.body);
            const hasUser = respObj.some(user => user.id === 111 && user.name === "John Doe");
            if (!hasUser) {
                pvg.fail("Created user not found in users list");
            }
            pvg.log.info(`Final users: ${JSON.stringify(respObj)}`);
        }
    });
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [50]:
!provengo run BookStoreDemo

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] INFO Input path: c:\temp\ProvengoDemo\BookStoreDemo
[RUN  ] INFO Preparing to run the model
[RUN>BUILD] INFO hello-world.js: Library 'REST' summoned automatically. Add "//@provengo summon rest" to the top of the file to explicitly summon it. Set the auto-summon configuration key to false to disable this behavior
[RUN>random] INFO B-program started
[RUN>random] INFO Selected: [GET {JS_Obj lib:"REST", method:"GET", url:"http://192.168.56.1:14701/users", headers:{JS_Obj Content-Type:"application/json"}, parameters:{JS_Obj }, expectedResponseCodes:[JS_Array ], callback:{JS_Obj }